# В данной LiveDemo демонстрируются варианты применения **Ripple** в реальных бизнес кейсах на примере денежный переводов.
## Демонстрируем замену системы переводов типа **Western Union**
 - не надо содержать офисы, проводим **KYC** онлайн с сопоставимой надежностью 
 - минимальные комиссии
 - высокие скорости
 - безопасность за счет применения встроенных решений: Ripple Escrow/Channels

 
### <span style="color:blue">Для простоты все примеры в тестовой сети</span>.

## Представим себе, что **Алиса** хочет отправить фиатный USD своему родственнику **Борису** в России, минуя ограничения SWIFT 
 - В качестве <span style="color:red">имеющегося ресурса</span> будем использовать имеющиеся **криптообменники** https://www.bestchange.ru/tether-trc20-to-sberbank.html - их очень много как <span style="color:red">замену банкам</span>
 - **Алиса** будет работать через Американский криптообменник и отправлть **USD**
 - **Борис** будет работать через российский криптообменник и получать **RUB** на Сбербанк
 - Конвертация будет происходить по курсу XRP/USD => XRP/RUB, другими словами **Ripple** используется как <span style="color:red">пул ликвидности</span>, что полностью согласуется с решниями сети:
     - https://ripple.com/solutions/crypto-liquidity/
     - https://www.exodus.com/news/ripple-on-demand-liquidity/
 - Пусть обменник **Алисы** называется **USAExchange**, а обменник, обслуживающий **Бориса** - **RUSCrypto**
 - **USAExchange** меняет Алисины USD на риплы и **депонирует** через смарт-контракт
 - **RUSCrypto** тоже депонирует средства и получает их обратно + комиссия после успешной сделки 

## Бизнес аспекты, которые требуют обсуждения:
  - Основная бизнес идея - создать конкурента **Western Union** или **Золотой короны** но работающий <span style="color:red">быстрее и дешевле, не зависеть от санкций и капризов банковской системы</span>
  - Крупный игрок может в <span style="color:red">перспективе **купить** проект</span>
  - Мы <span style="color:red">эксплуатируем</span> нежелание людей разбираться в крипте и криптокошельках. ВСе привыкли работать с <span style="color:red">фиатом.</span>
  - Мы будем прорабатывать <span style="color:red">гамму</span> стратегий для повышения вероятности успеха:
    1. **Вариант явной интеграции**: криптообменники должны согласиться на историю с депонированием (менять сложившуюся практику работы)
    2. **Вариант неявной интеграции**: обменник может послать нас на XXX с депонированием но тогда **USAExchange** переправляет рипплы на адрес смарт-контракта, а **RUSCrypto** будет получать на свой адрес.
    3. **Надо мотивировать** обменники работать через депонирование за счет более вкусных комиссий
    
Создать тестовые аккаунты Ripple из фразы можно тут: https://test.xrptoolkit.com/connect-wallet


In [120]:
import asyncio
import threading
import ipywidgets as widgets
from time import sleep
from xrpl.wallet import Wallet
from xrpl.clients import JsonRpcClient
from xrpl.account import get_account_info

##### UI #####
layout = widgets.Layout(width='auto', height='40px') #set width and height
button_defaults = widgets.Button(description="Использовать Кошельки по-умолчанию", button_style='primary', layout=layout)
button_customs = widgets.Button(description="У меня уже есть аккаунты в XRP сети", button_style='danger', layout=layout)
input_seed1 = widgets.Text(placeholder='SECRET-1')
input_seed1.layout.display = 'none'
input_seed2 = widgets.Text(placeholder='SECRET-2')
input_seed2.layout.display = 'none'
button_from_seeds = widgets.Button(description="Создать кошельки", button_style='success')
button_from_seeds.layout.display = 'none'
output = widgets.Output()
display(
    widgets.HBox([button_defaults, button_customs]), 
    widgets.HBox([input_seed1, input_seed2, button_from_seeds]),
    output
)
###############


##### TestNet ####
RPC_URL = 'https://s.altnet.rippletest.net:51234'
TEST_SEED1 = 'shCzgrD1fpECL8g3KfUDiWJSAy8Vj'
TEST_SEED2 = 'shQond3HBKuubYXs2Qfi7BTNQjCFo'

# USA - Обменник 1
USA_WALLET = None

# RUS - Обменник 2
RUS_WALLET = None

def init_wallets(usa_seed: str, rus_seed: str):
    global USA_WALLET
    global RUS_WALLET
    try:
        sleep(0.1)
        client = JsonRpcClient(RPC_URL)
        
        # init Wallet from SEED
        USA_WALLET = Wallet(usa_seed, sequence=0)
        # Load info from Ledger
        account_data = get_account_info(USA_WALLET.classic_address, client).result["account_data"]
        output.append_stdout(f'USA (Обменник 1)\n Address: {USA_WALLET.classic_address}\n Balance: {account_data["Balance"]}\n Sequence: {account_data["Sequence"]}\n Seed: {usa_seed}')
        
        # init Wallet from SEED
        RUS_WALLET = Wallet(rus_seed, sequence=0)
        # Load info from Ledger
        account_data = get_account_info(RUS_WALLET.classic_address, client).result["account_data"]
        output.append_stdout(f'\n\nRUS (Обменник 2)\n Address: {RUS_WALLET.classic_address}\n Balance: {account_data["Balance"]}\n Sequence: {account_data["Sequence"]}\n Seed: {rus_seed}')
        
        output.append_display_data(widgets.HTML('<span style="color:green">========== Wallets successfully created =============</span>'))
    except Exception as e:
        output.append_stderr(repr(e)) 
    
def on_click_defaults(b):
    output.clear_output()
    input_seed1.layout.display = 'none'
    input_seed2.layout.display = 'none'
    button_from_seeds.layout.display = 'none'
    th = threading.Thread(target=init_wallets, args=(TEST_SEED1, TEST_SEED2))
    th.start()
    
def on_create_from_seeds(b):
    seed1 = input_seed1.value
    seed2 = input_seed2.value
    th = threading.Thread(target=init_wallets, args=(seed1, seed2))
    th.start()
    
def on_click_customs(b):
    output.clear_output()
    input_seed1.layout.display = ''
    input_seed2.layout.display = ''
    button_from_seeds.layout.display = ''
    
button_defaults.on_click(on_click_defaults)
button_customs.on_click(on_click_customs)
button_from_seeds.on_click(on_create_from_seeds)

Output()

### Итак, когда оба обменника имеют аккаунты в блокчейн сети, мы можем реализовать перевод разными способами
### <span style="color:blue">Способ 1: Эскроу</span>
Создадим **Смарт-Контракт**, который в случае XRP реализует логику **Эскроу**, подробности тут https://xrpl.org/use-an-escrow-as-a-smart-contract.html 
**Гарантом** сделки, или **Оракулом** выступает <span style="color:red">наш сервис</span>

Алгоритм действия при денежном переводе:
 1. *Алиса* хочет перевести **100USD** *Борису*, курс обмена **доллар/рубль** ее устраивает, пусть например это будет эквивалент **50XRP**
 2. *Алиса* обращается в наш сервис и мы для нее нашли контрагента в США *USAExchange* и контрагента в РФ *RUSCrypto*
 3. Для *USAExchange* и *RUSCrypto* мы создаем 2 похожих Escrow, в которых депонируется **500XRP** (в полях Escrow контракта отправитель и получатель один адрес, Ripple это разрешает)
 4. Для разблокировки *Escrow* контракта создается **секрет**, как только *Борис* получит деньги, мы разморозим депонированные средства контрагентов, даже не притрагиваясь к ним
 5. Мы выступаем в качестве **Оракула**
 
 <span style="color:red">Кроме того, у смарт-контракта будет **время** исполнения</span> - если перевод не произведен, то депонированные средства разблокируются.
 <br/><span style="color:blue">Контрагенты будут зарабатывать на комиссии конвертации фиата в XRP</span>

In [139]:
import json
import math
import datetime
import ipywidgets as widgets
from xrpl.models import transactions
from xrpl.clients import JsonRpcClient
from xrpl.utils import xrp_to_drops, drops_to_xrp
from xrpl.transaction import safe_sign_and_autofill_transaction, send_reliable_submission
if USA_WALLET is None or RUS_WALLET is None:
    raise RuntimeError('Кошельки обменников не созданы, создайте их!!!')
    
RIPPLE_EPOCH_UNIX_OFFSET = 946684800
# Для примера пусть срок действия Эскроу блокировки будет 1 день
ESCROW_TIMEOUT = 24 * 60 * 60 
# Длдя примера зададим константами крпиптографические условие и его решение
DEMO_CONDITION = 'A0258020463F694BE607EB47E433EAA77428FB80097C6C4DC5117C548B65C557B31FE25E810120'
DEMO_FULFILMENT = 'A02280202861B2EFE96D6598AF034650E774F61CA74DBAB1C63AF8EC3A6617BDE46C123F'
ESCROW_TXN1_SEQUENCE = None
ESCROW_TXN2_SEQUENCE = None

######### UI ##############
layout = widgets.Layout(width='auto', height='40px') #set width and height
output_escrows = widgets.Output()
input_amount = widgets.IntText(value='50', description='Сумма XRP', layout=layout)
button_block_xrp = widgets.Button(description="Блокируем средства на счетах агентов", button_style='danger', layout=layout)
button_unblock_xrp = widgets.Button(description="Разблокируем средства на счетах агентов", button_style='success', layout=layout)
button_unblock_xrp.disabled = True
button_clear = widgets.Button(description="Очистить", layout=layout)
display(
    widgets.HBox([input_amount, button_block_xrp, button_unblock_xrp, button_clear]), 
    output_escrows
)
##########################


# Создаем Escrow транзакцию для контрагента
def create_escrow(contragent_address: str, amount: int, cancel_after: int, condition: str):
    create_escrow_txn = transactions.EscrowCreate(
        account=contragent_address,
        amount=xrp_to_drops(amount),
        destination=contragent_address,
        cancel_after=cancel_after,
        condition=condition
    )
    output_escrows.append_stdout(f'\n===== Escrow for Address: {contragent_address} ==========\n')
    output_escrows.append_stdout(json.dumps(create_escrow_txn.to_xrpl(), indent=2, sort_keys=True))
    return create_escrow_txn

# Контрагент подписывает транзакцию
def sign_escrow(wallet, escrow_txn):
    client = JsonRpcClient(RPC_URL)
    signed_txn = safe_sign_and_autofill_transaction(escrow_txn, wallet, client)
    output_escrows.append_stdout(f'\n===== Signed By Address: {wallet.classic_address} ==========\n')
    output_escrows.append_stdout(json.dumps(signed_txn.to_xrpl(), indent=2, sort_keys=True))
    return signed_txn


# Контрагенты прислали нам подписанные транзакции для повторной проверки
def apply_escrow_txns(signed_txn1, signed_txn2):
    client = JsonRpcClient(RPC_URL)
    txn1_response = send_reliable_submission(signed_txn1, client)
    txn2_response = send_reliable_submission(signed_txn2, client)
    output_escrows.append_stdout('\n===== TXN[1] ==========\n')
    output_escrows.append_stdout(json.dumps(txn1_response.result, indent=2, sort_keys=True))
    if txn1_response.result['meta']['TransactionResult'] != 'tesSUCCESS':
        raise RuntimeError('Error to Apply Excrow TXN[1]')
    output_escrows.append_stdout('\n===== TXN[2] ==========\n')
    output_escrows.append_stdout(json.dumps(txn2_response.result, indent=2, sort_keys=True))
    if txn2_response.result['meta']['TransactionResult'] != 'tesSUCCESS':
        raise RuntimeError('Error to Apply Excrow TXN[2]')
    
    
def block_ripples(amount: int):
    global ESCROW_TXN1_SEQUENCE
    global ESCROW_TXN2_SEQUENCE
    button_block_xrp.disabled = True
    try:
        try:
            sleep(0.3)
            cancel_after = math.floor(datetime.datetime.now().timestamp()) + (24 * 60 * 60) - RIPPLE_EPOCH_UNIX_OFFSET

            output_escrows.append_display_data(widgets.HTML('<span style="color:red">========== Create Escrows =============</span>'))
            usa_escrow_txn = create_escrow(USA_WALLET.classic_address, amount, cancel_after, DEMO_CONDITION)
            rus_escrow_txn = create_escrow(RUS_WALLET.classic_address, amount, cancel_after, DEMO_CONDITION)

            output_escrows.append_display_data(widgets.HTML('<span style="color:red">========== Sign Escrows by participants =============</span>'))
            usa_escrow_txn_sign = sign_escrow(USA_WALLET, usa_escrow_txn)
            ESCROW_TXN1_SEQUENCE = usa_escrow_txn_sign.sequence
            rus_escrow_txn_sign = sign_escrow(RUS_WALLET, rus_escrow_txn)
            ESCROW_TXN2_SEQUENCE = rus_escrow_txn_sign.sequence
            
            output_escrows.append_display_data(widgets.HTML('<span style="color:red">========== Check and Submit to XRP Ledger =============</span>'))
            apply_escrow_txns(usa_escrow_txn_sign, rus_escrow_txn_sign)
            
        except Exception as e:
            output_escrows.append_stderr(repr(e))
        else:
            button_unblock_xrp.disabled = False
    finally:
        button_block_xrp.disabled = False
        
        
def unblock_ripples():
    global ESCROW_TXN1_SEQUENCE
    global ESCROW_TXN2_SEQUENCE
    button_unblock_xrp.disabled = True
    try:
        try:
            
            client = JsonRpcClient(RPC_URL)
            for wallet, txn_sequence in [(USA_WALLET, ESCROW_TXN1_SEQUENCE), (RUS_WALLET, ESCROW_TXN2_SEQUENCE)]:
                finish_txn = transactions.EscrowFinish(
                    account=wallet.classic_address,
                    owner=wallet.classic_address,
                    offer_sequence=txn_sequence,
                    condition=DEMO_CONDITION,
                    fulfillment=DEMO_FULFILMENT
                )
                
                signed_finish_tx = safe_sign_and_autofill_transaction(finish_txn, wallet, client)
                tx_response = send_reliable_submission(signed_finish_tx, client)
                output_escrows.append_display_data(widgets.HTML(f'<span style="color:red">========== Address: {wallet.classic_address} =============</span>'))
                output_escrows.append_stdout(json.dumps(tx_response.result, indent=2, sort_keys=True))
        except Exception as e:
            output_escrows.append_stderr(repr(e))
    finally:
        button_unblock_xrp.disabled = False
        
    

def on_block_click(b):
    ESCROW_TXN1_SEQUENCE = None
    ESCROW_TXN2_SEQUENCE = None
    output_escrows.clear_output()
    button_unblock_xrp.disabled = True
    amount = input_amount.value
    th = threading.Thread(target=block_ripples, args=(amount,))
    th.start()
    
    
def on_unblock_click(b):
    output_escrows.clear_output()
    th = threading.Thread(target=unblock_ripples, args=())
    th.start()
    
    
button_block_xrp.on_click(on_block_click)
button_unblock_xrp.on_click(on_unblock_click)
button_clear.on_click(lambda f: output_escrows.clear_output())

Output()

### <span style="color:blue">Способ 2: Электронные чеки</span>
Ripple позволяет обкешивать эл.чеки, технология описана тут https://xrpl.org/checks.html
Этот кейс возможен, если банк *Алисы* работает с криптой и у *Алисы* в этом банке есть криптокошелек. Рабочий вариант - **Revolut** который с работает с криптокошельками.
В этом случае *Алиса* через мобильный банк просит выпустить чек, банк блокирует на ее счете **USD** и переводит по курсу в **XRP** и на заблокированные **XRP** выпускает чек. 

У этого кейса есть плюсы и минусы:
 1. **Минус**: в момент создания чека в качестве адресата должен быть указан адрес агента *Бориса*, *Борис* должен **доверять** своему контрагенту и скорее всего это тоже должен быть **банк**
 2. **Плюс**: чеки очень просты в реализации, но ребуют поддержки работы с XRP обоими банками
 
#### *Опционально:* Чеки можно реализовать через Escrow
>Этот сценарий **применим**, если отправитель работает через **свой Банк, а получатель работает через сторонний обменник: средства блокируются на стороне *Алисы*, но адресатом в Escrow указан обменник, который должен выдать средства *Борису*
 
### <span style="color:blue">Способ 3: Платежный канал</span>
Эта технология подробно описана тут: https://xrpl.org/payment-channels.html
Плюсы и минусы:
 1. **Плюс**: позволяет переводить XRP по частям, если нет полного доверия контрагенту на стороне *Бориса*
 2. **Плюс**: платежи можно разбивать на очень мелкие, не боясь, что **комиссия съест** общую сумму
 3. **Плюс**: если на стороне *Бориса* банк блокирует перевод сразу крупной суммы, можно разбить на более мелкие или зачтислять на карты разных банков
 2. **Минус**: требует дополнительного программирования и интеграции в IT систему банка/обменника. Можг